In [91]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

In [92]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(37) # Seed 고정

In [93]:
train_df = pd.read_csv('/Users/sola/Downloads/open/train.csv')
test_df = pd.read_csv('/Users/sola/Downloads/open/test.csv')

In [94]:
train_df.isnull().sum()

PRODUCT_ID      0
Y_Class         0
Y_Quality       0
TIMESTAMP       0
LINE            0
             ... 
X_2871        499
X_2872        598
X_2873        598
X_2874        598
X_2875        598
Length: 2881, dtype: int64

In [95]:
# 결측치에 평균 넣어주기
train_df=train_df.fillna(train_df.mean())
test_df=test_df.fillna(test_df.mean())

/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_2879/1987006743.py:2: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  train_df=train_df.fillna(train_df.mean())
/var/folders/js/792bsrvx24lcg4nrdgy7k5br0000gn/T/ipykernel_2879/1987006743.py:3: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  test_df=test_df.fillna(test_df.mean())


In [96]:
train_df.isnull().sum()

PRODUCT_ID      0
Y_Class         0
Y_Quality       0
TIMESTAMP       0
LINE            0
             ... 
X_2871          0
X_2872        598
X_2873        598
X_2874        598
X_2875        598
Length: 2881, dtype: int64

In [97]:
test_df.isnull().sum()

PRODUCT_ID      0
TIMESTAMP       0
LINE            0
PRODUCT_CODE    0
X_1             0
               ..
X_2871          0
X_2872          0
X_2873          0
X_2874          0
X_2875          0
Length: 2879, dtype: int64

In [98]:
train_df = train_df.fillna(0)
test_df = test_df.fillna(0)

In [99]:
train_df.isnull().sum()

PRODUCT_ID    0
Y_Class       0
Y_Quality     0
TIMESTAMP     0
LINE          0
             ..
X_2871        0
X_2872        0
X_2873        0
X_2874        0
X_2875        0
Length: 2881, dtype: int64

In [100]:
# 프로더그 코드별로 구분시켜서 학습시키려고..

# atr = train_df[train_df["PRODUCT_CODE"]=="A_31"]
# ttr = train_df[train_df["PRODUCT_CODE"]=="T_31"]
# otr = train_df[train_df["PRODUCT_CODE"]=="O_31"]

In [101]:
train_x = train_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP', 'Y_Class', 'Y_Quality'])
train_y = train_df['Y_Class']

test_x = test_df.drop(columns=['PRODUCT_ID', 'TIMESTAMP'])
# test_y = test_df['Y_Class']

In [102]:
test_x.columns

Index(['LINE', 'PRODUCT_CODE', 'X_1', 'X_2', 'X_3', 'X_4', 'X_5', 'X_6', 'X_7',
       'X_8',
       ...
       'X_2866', 'X_2867', 'X_2868', 'X_2869', 'X_2870', 'X_2871', 'X_2872',
       'X_2873', 'X_2874', 'X_2875'],
      dtype='object', length=2877)

In [70]:
# 결측치에 평균 넣어주기
# for i in range(1,2876):
#     co = "X_" + str(i)
#     train_df.loc[train_df[co].isna(), co]= train_df[co].mean()
#
# for i in range(1,2876):
#     co = "X_" + str(i)
#     test_df.loc[test_df[co].isna(), co]= test_df[co].mean()

In [103]:
train_x.isnull().sum()

LINE            0
PRODUCT_CODE    0
X_1             0
X_2             0
X_3             0
               ..
X_2871          0
X_2872          0
X_2873          0
X_2874          0
X_2875          0
Length: 2877, dtype: int64

In [104]:
# # 결측치에 0을 넣어줬네..
# train_x = train_x.fillna(0)
# test_x = test_x.fillna(0)

In [105]:
# 범주형을 수치형으로 변환하는 과정
qual_col = ['LINE', 'PRODUCT_CODE']

for i in qual_col:
    le = LabelEncoder()
    le = le.fit(train_x[i])
    train_x[i] = le.transform(train_x[i])

    for label in np.unique(test_x[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test_x[i] = le.transform(test_x[i])
print('Done.')

Done.


In [106]:
RF = RandomForestClassifier(n_jobs = -1 ,random_state=37)

In [107]:
# 교차검증
from sklearn.model_selection import cross_validate
scores = cross_validate(RF, train_x, train_y, return_train_score=True, n_jobs=-1)

In [108]:
print(np.mean(scores['train_score']), np.mean(scores['test_score']))

1.0 0.5599439775910364


이제 fitting 시키기

In [109]:
# fit
RF.fit(train_x, train_y)

RandomForestClassifier(n_jobs=-1, random_state=37)

In [88]:
train_x.isnull().sum()

LINE            0
PRODUCT_CODE    0
X_1             0
X_2             0
X_3             0
               ..
X_2871          0
X_2872          0
X_2873          0
X_2874          0
X_2875          0
Length: 2877, dtype: int64

In [114]:
preds = RF.predict(test_x)

In [115]:
preds

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1,
       1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1,
       1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1,
       0, 1, 2, 1, 2, 2, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 1, 1, 2, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 0, 1,
       1, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 2, 1, 1, 1, 1, 1, 2, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 2, 2, 1, 1, 2, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 2, 0, 0, 0, 2, 2, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2,
       1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1,

In [116]:
submit = pd.read_csv('/Users/sola/Downloads/open/sample_submission.csv')

In [117]:
submit['Y_Class'] = preds

In [118]:
submit.to_csv('/Users/sola/Downloads/open/baseline_submission2.csv', index=False)

결측치에 모두 0을 넣으면 정확도는 0.5495720225, 136등임...

결측치에 평균값넣고 아무것도 없는 컬럼에는 0을 넣으면 정확도는 0.4457953394임...
왜 더 내려갔누...

In [ ]:
# from sklearn.metrics import accuracy_score
# accuracy = accuracy_score(test_y, preds)
# print('랜덤 포레스트 정확도: {:.4f}'.format(accuracy))